In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Ready_for_analysis').getOrCreate()



In [2]:
df = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/MPI/*',
    header=True)


In [3]:
df.createOrReplaceTempView('MPI_test')


In [4]:
df.show()

+-----------+-----------------------------+------------------------------------+------------------+--------------------------------+-------------------------+-------------------------+-------------------------+
|scrape_date|master_price_api_market_price|master_price_api_market_price_last3m|         item_code|master_priceapi_competitor_count|master_item_main_category|master_item_sub_category1|master_item_sub_category2|
+-----------+-----------------------------+------------------------------------+------------------+--------------------------------+-------------------------+-------------------------+-------------------------+
| 2020-12-15|                        160.7|                               160.7|000000001000150797|                             5.0|                 LEUCHTEN|            INNENLEUCHTEN|              BADLEUCHTEN|
| 2020-12-15|                       248.14|                              248.14|000000001000191998|                             5.0|                      BA

In [9]:
df2 = spark.sql('SELECT scrape_date, item_code, master_price_api_market_price_last3m as mpi,master_priceapi_competitor_count,master_item_main_category,master_item_sub_category1 from MPI_test ORDER BY scrape_date DESC')
#df2.show()
df2.createOrReplaceTempView('MPI_trend_list')

In [13]:
#df_items_wo_full_hist =spark.sql(""" SELECT item_code
#                                    FROM MPI_trend_list
#                                    GROUP BY item_code, master_item_main_category
#                                    HAVING COUNT(scrape_date)  == 12 """)

+------------------+
|item_code         |
+------------------+
|000000001000176199|
|000000001000136999|
|000000001000209178|
|000000001000098912|
|000000001000099240|
|000000001000075681|
|000000001000208787|
|000000001000087725|
|000000001000178419|
|000000001000220011|
|000000001000178296|
|000000001000163180|
|000000001000117209|
|000000001000171869|
|000000001000198379|
|000000001000146704|
|000000001000194826|
|000000001000141714|
|000000001000191839|
|000000001000047506|
|000000001000146273|
|000000001000045347|
|000000001000209340|
|000000001000176187|
|000000001000045552|
|000000001000205473|
|000000001000215904|
|000000001000179072|
|000000001000134968|
|000000001000190357|
|000000001000147970|
|000000001000097828|
|000000001000178287|
|000000001000087349|
|000000001000101053|
|000000001000168006|
|000000001000166070|
|000000001000165186|
|000000001000111711|
|000000001000193239|
|000000001000179626|
|000000001000149170|
|000000001000198295|
|000000001000163736|
|000000001000

In [31]:
df_per_cat = spark.sql("""SELECT scrape_date, avg(mpi) as mpi, master_item_main_category
                          from MPI_trend_list
                          where item_code in (SELECT item_code
                                    FROM MPI_trend_list
                                    GROUP BY item_code, master_item_main_category
                                    HAVING COUNT(scrape_date)  == 12 )
                          GROUP BY master_item_main_category, scrape_date
                          ORDER BY scrape_date,master_item_main_category""")

df_per_cat.show(1000,False)


+-----------+------------------+-------------------------+
|scrape_date|mpi               |master_item_main_category|
+-----------+------------------+-------------------------+
|2020-09-29 |377.5331343283582 |BABY                     |
|2020-09-29 |274.50396082698563|BAD                      |
|2020-09-29 |582.4469801980197 |BETTEN                   |
|2020-09-29 |38.460814393939366|BOUTIQUE                 |
|2020-09-29 |284.85772277227693|BÜRO                     |
|2020-09-29 |180.2316949152542 |ESSZIMMERSTÜHLE&BÄNKE    |
|2020-09-29 |457.1660606060609 |ESSZIMMERTISCHE          |
|2020-09-29 |243.94440092165874|FLURMÖBEL                |
|2020-09-29 |167.15721739130436|GARTEN                   |
|2020-09-29 |82.21000000000001 |HAUSHALT                 |
|2020-09-29 |47.88989680426095 |HEIMTEXTILIEN            |
|2020-09-29 |206.78411347517743|KINDER                   |
|2020-09-29 |1067.4644444444443|KÜCHE                    |
|2020-09-29 |74.84208911614336 |LEUCHTEN                

In [32]:
main_cat = spark.sql('SELECT DISTINCT master_item_main_category as category from MPI_test')
#main_cat.show()
cat_list = main_cat.rdd.map(lambda x: x.category).collect()
main_subcat = spark.sql('SELECT DISTINCT master_item_sub_category1 from MPI_test')
subcat_list = main_subcat.rdd.map(lambda x: x.master_item_sub_category1).collect()


In [33]:
pd_df = df2.toPandas()
pd_per_cat = df_per_cat.toPandas()

In [34]:
print(cat_list)
print(subcat_list)
cat_list_copy = cat_list

['TEPPICHE & BODENBELÄGE', 'HAUSHALT', 'ESSZIMMERTISCHE', 'LEUCHTEN', 'POLSTER', 'BÜRO', 'KINDER', 'FLURMÖBEL', 'SCHLAFZIMMERMÖBEL', 'WOHNEN', 'GARTEN', 'KÜCHE', 'BOUTIQUE', 'BABY', 'HEIMTEXTILIEN', 'MATRATZEN&LATTENROSTE', 'BETTEN', 'BAD', 'ESSZIMMERSTÜHLE&BÄNKE']
['BADEMATTEN & BADTEPPICHE', 'KOMPLETTPROGRAMME & SETS', 'GARTENACCESSOIRES', 'KINDERTEPPICHE', 'GARTENZUBEHÖR', 'SITZWÜRFEL & POLSTERHOCKER', 'NACHTTISCHE', 'BETTLAKEN', 'BÜRO-KOMPLETTSETS', 'HAUSHALT', 'NACHTKOMMODEN', 'SAISONARTIKEL WEIHNACHTEN', 'BADACCESSOIRES', 'HIMMELBETTEN', 'GARTENTISCHE', 'SCHLAFZIMMERSETS', 'HANDTÜCHER', 'FUß- & SAUBERLAUFMATTEN', 'KAMINE', 'SCHLAFSOFAS', 'GARDINEN & VORHÄNGE', 'WANDEKORATION', 'BÜROSCHRÄNKE', 'ESSZIMMERSTÜHLE', 'KURZFLORTEPPICHE', 'BETTDECKEN & KOPFKISSEN', 'RECAMIEREN', 'AUFBEWAHRUNG', 'RELAXLIEGEN', 'ZUBEHÖR', 'TISCHE', 'BETTBÄNKE&BETTTRUHEN', 'TISCHWÄSCHE', 'GARTENSTÜHLE', 'BÄNKE', 'HOCHSTÜHLE & ZUBEHÖR', 'BETTGESTELLE', 'BETTEN & ZUBEHÖR', 'SPIEGEL', 'OUTDOOR-TEPPICHE', 'DUFT

In [35]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 5000)


In [36]:
#print(pd_df['scrape_date'].max())
#print(pd_df['scrape_date'].min())
def modify_column(data_frame):
    data_frame['date_delta'] = (data_frame['scrape_date'] - data_frame['scrape_date'].min())  / np.timedelta64(7,'D')
    data_frame['log_mpi']= np.log(data_frame['mpi'])
    return data_frame

pd_df = modify_column(pd_df)
pd_per_cat = modify_column(pd_per_cat)


In [10]:
#THINK ABOUT A MINIMUM NUMBER OF POINTS TO MAKE THE PLOT REASONABLE
#pd_df_one_item = pd_df[pd_df['item_code'] == '000000001000137415']
#pd_df_one_item.plot('date_delta','mpi',style = 'o')
#pd_df_one_item.head(20)

In [38]:
def run_model(data_frame=pd_df):
    return smf.ols("mpi ~ date_delta + C(item_code)",data_frame).fit()

def run_model_per_cat(data_frame=pd_per_cat):
    return smf.ols("mpi ~ date_delta",data_frame).fit()

In [39]:
print(cat_list)
cat_list.remove('HEIMTEXTILIEN')
print(cat_list)

['TEPPICHE & BODENBELÄGE', 'HAUSHALT', 'ESSZIMMERTISCHE', 'LEUCHTEN', 'POLSTER', 'BÜRO', 'KINDER', 'FLURMÖBEL', 'SCHLAFZIMMERMÖBEL', 'WOHNEN', 'GARTEN', 'KÜCHE', 'BOUTIQUE', 'BABY', 'HEIMTEXTILIEN', 'MATRATZEN&LATTENROSTE', 'BETTEN', 'BAD', 'ESSZIMMERSTÜHLE&BÄNKE']
['TEPPICHE & BODENBELÄGE', 'HAUSHALT', 'ESSZIMMERTISCHE', 'LEUCHTEN', 'POLSTER', 'BÜRO', 'KINDER', 'FLURMÖBEL', 'SCHLAFZIMMERMÖBEL', 'WOHNEN', 'GARTEN', 'KÜCHE', 'BOUTIQUE', 'BABY', 'MATRATZEN&LATTENROSTE', 'BETTEN', 'BAD', 'ESSZIMMERSTÜHLE&BÄNKE']


In [46]:
def get_model_dict(Cat_list,data_frame, which_model='CAT'):
    models = {}
    for cat in Cat_list:
        print('Category = ' + cat)
        pd_df_one_cat = data_frame[data_frame['master_item_main_category'] == cat]
        if which_model == 'CAT':
            print('Running model per category')
            models[cat] = run_model_per_cat(pd_df_one_cat)
        elif which_model == 'ITEM':
            models[cat] = run_model(pd_df_one_cat)
    return models

#print(models.keys())


In [47]:
model_per_cat = get_model_dict(cat_list,pd_per_cat)

Category = TEPPICHE & BODENBELÄGE
Running model per category
Category = HAUSHALT
Running model per category
Category = ESSZIMMERTISCHE
Running model per category
Category = LEUCHTEN
Running model per category
Category = POLSTER
Running model per category
Category = BÜRO
Running model per category
Category = KINDER
Running model per category
Category = FLURMÖBEL
Running model per category
Category = SCHLAFZIMMERMÖBEL
Running model per category
Category = WOHNEN
Running model per category
Category = GARTEN
Running model per category
Category = KÜCHE
Running model per category
Category = BOUTIQUE
Running model per category
Category = BABY
Running model per category
Category = MATRATZEN&LATTENROSTE
Running model per category
Category = BETTEN
Running model per category
Category = BAD
Running model per category
Category = ESSZIMMERSTÜHLE&BÄNKE
Running model per category


In [52]:
def print_results(model_dict):
        for categ in model_dict.keys():
            print(categ)
            m     = model_dict[categ].params[-1].round(2)
            m_err = model_dict[categ].bse[-1].round(2)
            result = 'Trend = ' + str(m) + ' +/- ' + str(m_err)
            #print(model_dict[categ].summary())
            print(result)

In [54]:
print_results(model_per_cat)

TEPPICHE & BODENBELÄGE
Trend = -0.19 +/- 0.06
HAUSHALT
Trend = -1.12 +/- 0.16
ESSZIMMERTISCHE
Trend = -0.92 +/- 0.16
LEUCHTEN
Trend = 0.22 +/- 0.03
POLSTER
Trend = 0.71 +/- 0.23
BÜRO
Trend = -0.24 +/- 0.07
KINDER
Trend = -0.4 +/- 0.07
FLURMÖBEL
Trend = 0.04 +/- 0.08
SCHLAFZIMMERMÖBEL
Trend = -0.17 +/- 0.08
WOHNEN
Trend = -0.52 +/- 0.11
GARTEN
Trend = 0.13 +/- 0.14
KÜCHE
Trend = 0.99 +/- 0.6
BOUTIQUE
Trend = 0.0 +/- 0.01
BABY
Trend = -0.66 +/- 0.17
MATRATZEN&LATTENROSTE
Trend = -0.21 +/- 0.22
BETTEN
Trend = -1.09 +/- 0.2
BAD
Trend = -0.2 +/- 0.18
ESSZIMMERSTÜHLE&BÄNKE
Trend = -0.73 +/- 0.11


In [57]:
#To investigate HEIMTEXTILIEN
sub_models = {}

for subcat in subcat_list:
    print('Category = ' + subcat)
    pd_df_one_subcat = pd_df[pd_df['master_item_sub_category1'] == subcat]
    sub_models[subcat] = run_model(pd_df_one_subcat)
print(sub_models.keys())


Category = BADEMATTEN & BADTEPPICHE
Category = KOMPLETTPROGRAMME & SETS
Category = GARTENACCESSOIRES
Category = KINDERTEPPICHE
Category = GARTENZUBEHÖR
Category = SITZWÜRFEL & POLSTERHOCKER
Category = NACHTTISCHE
Category = BETTLAKEN
Category = BÜRO-KOMPLETTSETS
Category = HAUSHALT
Category = NACHTKOMMODEN
Category = SAISONARTIKEL WEIHNACHTEN
Category = BADACCESSOIRES
Category = HIMMELBETTEN
Category = GARTENTISCHE
Category = SCHLAFZIMMERSETS
Category = HANDTÜCHER
Category = FUß- & SAUBERLAUFMATTEN
Category = KAMINE
Category = SCHLAFSOFAS
Category = GARDINEN & VORHÄNGE
Category = WANDEKORATION
Category = BÜROSCHRÄNKE
Category = ESSZIMMERSTÜHLE
Category = KURZFLORTEPPICHE
Category = BETTDECKEN & KOPFKISSEN
Category = RECAMIEREN
Category = AUFBEWAHRUNG
Category = RELAXLIEGEN
Category = ZUBEHÖR
Category = TISCHE
Category = BETTBÄNKE&BETTTRUHEN
Category = TISCHWÄSCHE
Category = GARTENSTÜHLE
Category = BÄNKE
Category = HOCHSTÜHLE & ZUBEHÖR
Category = BETTGESTELLE
Category = BETTEN & ZUBEHÖR

In [58]:
for subcateg in sub_models.keys():
    print(subcateg)
    m     = sub_models[subcateg].params[-1].round(2)
    m_err = sub_models[subcateg].bse[-1].round(2)
    result = 'Trend = ' + str(m) + ' +/- ' + str(m_err)
    #print(models[categ].summary())
    print(result)

BADEMATTEN & BADTEPPICHE
Trend = -0.04 +/- 0.01
KOMPLETTPROGRAMME & SETS
Trend = 0.18 +/- 0.55
GARTENACCESSOIRES
Trend = 0.14 +/- 0.06
KINDERTEPPICHE
Trend = -0.35 +/- 0.06
GARTENZUBEHÖR
Trend = -0.03 +/- 0.04
SITZWÜRFEL & POLSTERHOCKER
Trend = 0.11 +/- 0.06
NACHTTISCHE
Trend = -0.46 +/- 0.12
BETTLAKEN
Trend = -0.13 +/- 0.01
BÜRO-KOMPLETTSETS
Trend = -2.24 +/- 2.08
HAUSHALT
Trend = -1.12 +/- 0.14
NACHTKOMMODEN
Trend = 0.15 +/- 0.06
SAISONARTIKEL WEIHNACHTEN
Trend = 0.07 +/- 0.02
BADACCESSOIRES
Trend = 0.0 +/- 0.01
HIMMELBETTEN
Trend = -4.92 +/- 4.71
GARTENTISCHE
Trend = 0.22 +/- 0.18
SCHLAFZIMMERSETS
Trend = 1.04 +/- 0.59
HANDTÜCHER
Trend = -0.07 +/- 0.01
FUß- & SAUBERLAUFMATTEN
Trend = -0.14 +/- 0.02
KAMINE
Trend = -11.97 +/- 2.51
SCHLAFSOFAS
Trend = 0.48 +/- 0.39
GARDINEN & VORHÄNGE
Trend = 0.13 +/- 0.01
WANDEKORATION
Trend = -0.19 +/- 0.02
BÜROSCHRÄNKE
Trend = -1.0 +/- 0.23
ESSZIMMERSTÜHLE
Trend = -0.74 +/- 0.06
KURZFLORTEPPICHE
Trend = -0.69 +/- 0.06
BETTDECKEN & KOPFKISSEN
Trend =